In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/Users/meruozhu/Downloads/MP_data/MP_codes/MP')
from DPM.task_free_continual_learning.method_provide_model_test import Task_free_continual_learning_provide_model
from DPM.task_free_continual_learning.sampler_provide_model import Sampler
from edbn.Methods.SDL.sdl import transform_data

/Users/meruozhu/opt/anaconda3/envs/pmGrad/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import time
import pandas as pd
import numpy as np
from scipy.stats import percentileofscore
from Data.data import Data
from edbn.Utils.LogFile import LogFile
import edbn.Predictions.setting as setting
from edbn import Methods
from collections import deque
from itertools import islice
from collections import OrderedDict
from PrefixTreeCDD.PrefixTreeClass import PrefixTree
import PrefixTreeCDD.settings as settings
from PrefixTreeCDD.CDD import Window
from skmultiflow.drift_detection import ADWIN, PageHinkley
import math
from numpy import log as ln
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import utils as ku
import numpy.random as rn

## get data in logfile format, taken as input of sampler

In [20]:
file = '/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/Helpdesk_mini.csv'
data = pd.read_csv(file, low_memory=False)
timeformat = "%Y-%m-%d %H:%M:%S"
numEvents = data.shape[0]
print("Num events is {}".format(numEvents))

dataName = 'Helpdesk_mini'
log = LogFile(filename=file, delim=",", header=0, rows=None, time_attr="completeTime", trace_attr="case",
                    activity_attr='event', convert=False)
d = Data(dataName,
            LogFile(filename=file, delim=",", header=0, rows=None, time_attr="completeTime", trace_attr="case",
                    activity_attr='event', convert=False))
d.logfile.keep_attributes(['event', 'role', 'completeTime'])
m = Methods.get_prediction_method("SDL")
s = setting.STANDARD
trainPerc = 0.8
s.train_percentage = trainPerc * 100
# # #
d.prepare(s)
# d.create_batch("normal", timeformat)
is_written = 0

start_time = time.time()
# print(d.train.contextdata)
# print("Test Context Data")
# print(d.test_orig.contextdata)
# basic_model = m.train(d.train)

Num events is 300
PREPARE
CONVERT
PREPROCESSING: Converting event
PREPROCESSING: Converting role
PREPROCESSING: Converting case
K-CONTEXT
Create k-context: 10
SPLIT TRAIN-TEST
Train: 240
Test: 60


## change sampler to get similar output

In [65]:
alpha=1.0, 

ntasks=2
dim=4
discriminator_offset=0.01
distribution_offset=0.01
uniform_width=1.25
nsamples=500
ntrain=40
ntest=2
# seed
rn.seed(512)
stime=time.time()
tasks={}
        
# sample points for four tasks related to four quadrants
x, y, vals= transform_data(d.train, [a for a in d.train.attributes() if a != d.train.time and a != d.train.trace])
samples=[]
for i in range(len(d.test_orig.get_data())):
    sample = []
    for j in range(len(x)):
        # sample.append(np.array([x[j][i]]))
        sample.append(x[j][i])
    samples.append(np.array(sample))
for q in range(ntasks):
    if q == 0: # 1th task
        tasks[q]=np.asarray(samples[:150])
    elif q == 1: # 2nd task
        tasks[q]=np.asarray(samples[150:300])

In [66]:
samples[50]

array([5, 6, 1, 0, 0, 0, 0, 0, 0, 0, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0])

In [67]:
inputs={}
labels={}

# Step 3: Sample correct distribution given certain alpha
for q in range(ntasks):
    inputs[q]=samples[:int(ntrain)]
    labels[q]=np.argmax(y,axis=1)[:int(ntrain)]

In [74]:
labels[0][1]

1

In [73]:
inputs[1][0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [61]:
np.array(inputs[0][0])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [75]:
test_inputs={}
test_labels={}
for q in range(ntasks):
    inputs[q]=samples[:int(ntest)]
    labels[q]=np.argmax(y,axis=1)[:int(ntest)]

In [21]:
s = Sampler(data=d)

In [22]:
s.inputs.keys()

dict_keys([0, 1])

In [135]:
learning_object=Task_free_continual_learning_provide_model(verbose=False,
                                                        seed=123,
                                                        dev='cpu',
                                                        dim=4,
                                                        hidden_units=100,
                                                        learning_rate=0.005,
                                                        ntasks=2,
                                                        gradient_steps=5,
                                                        loss_window_length=5,
                                                        loss_window_mean_threshold=0.2,
                                                        loss_window_variance_threshold=0.1,                                                         
                                                        MAS_weight=0.5,
                                                        recent_buffer_size=20,
                                                        hard_buffer_size=5, model = basic_model)

TypeError: __init__() got an unexpected keyword argument 'verbose'

In [133]:
learning_object.method(s,use_hard_buffer=True,continual_learning=True)

x [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] y 1


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 20 array(s), but instead got the following list of 1 arrays: [array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...

In [131]:
x=[0,0]
np.asarray(x)

array([0, 0])

## get correct test dataset

In [40]:
s = Sampler(data=d)

In [41]:
print(len(s.test_inputs[1]))
s.test_inputs[1][0]

60


array([5, 6, 7, 6, 1, 0, 0, 0, 0, 0, 6, 6, 6, 6, 6, 0, 0, 0, 0, 0])

In [34]:
len(s.inputs[1])

240

In [44]:
d.train.get_data()

,level_0,index,event_Prev9,role_Prev9,completeTime_Prev9,case_Prev9,event_Prev8,role_Prev8,completeTime_Prev8,case_Prev8,...,completeTime_Prev1,case_Prev1,event_Prev0,role_Prev0,completeTime_Prev0,case_Prev0,event,role,completeTime,case
0,0.0,191,0,0,0,59,0,0,0,59,...,0,59,0,0,0,59,1,6,2010/01/13 07:40:25.000,59
1,1.0,143,0,0,0,43,0,0,0,43,...,0,43,0,0,0,43,1,6,2010/01/13 11:26:04.000,43
2,2.0,234,0,0,0,70,0,0,0,70,...,0,70,0,0,0,70,1,6,2010/01/13 11:30:37.000,70
3,3.0,58,0,0,0,17,0,0,0,17,...,0,17,0,0,0,17,1,6,2010/01/13 12:09:31.000,17
4,4.0,220,0,0,0,67,0,0,0,67,...,0,67,0,0,0,67,1,6,2010/01/13 16:25:25.000,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,235.0,113,0,0,0,33,0,0,0,33,...,0,33,0,0,0,33,1,6,2010/02/25 14:07:33.000,33
236,236.0,20,0,0,0,6,0,0,0,6,...,2010/02/08 14:12:00.000,6,7,5,2010/02/08 14:12:28.000,6,6,3,2010/02/25 15:56:34.000,6
237,237.0,204,0,0,0,61,0,0,0,61,...,2010/02/04 07:22:54.000,61,7,6,2010/02/04 08:07:26.000,61,6,6,2010/02/26 07:53:59.000,61
238,238.0,205,0,0,0,61,0,0,0,61,...,2010/02/04 08:07:26.000,61,6,6,2010/02/26 07:53:59.000,61,5,6,2010/02/26 07:54:05.000,61
